In [ ]:
import numpy as np

In [ ]:
def house(x):
    n=len(x); mu=np.linalg.norm(x); v=np.copy(x)
    if mu:
        beta=x[0]+(1.0 if x[0] >= 0 else -1.0)*mu
        v[1:]/=beta
    v[0]=1.0
    return v


In [ ]:
def row_house(A,v):
    beta = -2.0 / np.dot(v,v)
    w = beta * np.matmul(A.T,v)
    A+=np.outer(v,w)


In [ ]:
def house_rev_accum(A,B):
    m,n=A.shape
    v=np.zeros(m)
    for j in range(n-1,-1,-1):
        v[j]=1.0; v[j+1:]=A[j+1:,j]
        row_house(B[j:,j:],v[j:])


In [ ]:
def house_QR(A):
    m,n=A.shape
    for j in range(n):
        v=house(A[j:,j])
        row_house(A[j:,j:],v)
        if j<m-1:
            A[j+1:,j]=v[1:]
    return A



1. Решение СЛАУ методом Хаусхолдера, вычисление норм невязок, погрешностей:

In [ ]:
def solvehouse(A1,b):
    A=np.copy(A1)
    house_QR(A)
    m,n=A.shape
    Q=np.eye(m)
    house_rev_accum(A,Q)
    R=np.zeros((m,n))
    for j in range(n):
        R[:j+1,j]=A[:j+1,j]
    x=np.empty(A.shape[0])
    b = np.matmul(Q.T,b)
    for i in range(A.shape[0]-1,-1,-1):
        x[i]=b[i]/A[i][i]
        for j in range(A.shape[0]-1,i,-1):
            x[i]-=x[j]*A[i][j]/A[i][i]
    return x

A_1 = np.array([[ 127.8000,8.0300,1.4000,-2.3600],
      [0.2700,136.4000,-0.1600,-4.5500],
     [-3.8400,5.3700,-111.0000,1.5600],
     [-6.5300,6.7200,2.8800,47.2000]])
b_1 = np.array([-1008.64, 516.62, 394.56, 353.68])
x_1 = np.array([-8,4,-3,6], dtype = 'float64') #точное решение

A_2 = np.array([[3.8970,-3.8940,19.0620,27.2580],
     [29.1600,-29.1570,158.9520,198.7160],
      [0.9720,-0.9720,4.7610,6.8040],
      [2.9160,-2.9160,16.5900,19.6430]])
b_2 = np.array([-17.2050, -426.2380, -4.3470, -55.3360])
x_2 = np.array([13,14,-15,10], dtype = 'float64') #точное решение


x_11 = solvehouse(A_1,b_1)
x_22 = solvehouse(A_2,b_2)
print(x_11)
print(x_22)

print('1-норма  невязки  для первой матрицы = ', np.linalg.norm((np.dot(A_1,x_11)-b_1),1))
print('1-норма невязки  для второй матрицы = ', np.linalg.norm((np.dot(A_2,x_22)-b_2),1))
print('inf-норма  невязки  для первой матрицы = ', np.linalg.norm((np.dot(A_1,x_11)-b_1), np.inf))
print('inf-норма невязки  для второй матрицы = ', np.linalg.norm((np.dot(A_2,x_22)-b_2), np.inf))
print('абсолютная погрешность для первой матрицы = ', np.linalg.norm(x_11-x_1))
print('абсолютная погрешность для второй матрицы  = ', np.linalg.norm(x_22-x_2))
print('относительная погрешность для первой матрицы = ', np.linalg.norm(x_11-x_1)/np.linalg.norm(x_1))
print('относительная погрешность для второй матрицы = ', np.linalg.norm(x_22-x_2)/np.linalg.norm(x_1))




[-8.  4. -3.  6.]
[ 13.  14. -15.  10.]
1-норма  невязки  для первой матрицы =  2.8421709430404007e-13
1-норма невязки  для второй матрицы =  8.162359677044151e-13
inf-норма  невязки  для первой матрицы =  1.1368683772161603e-13
inf-норма невязки  для второй матрицы =  6.252776074688882e-13
абсолютная погрешность для первой матрицы =  0.0
абсолютная погрешность для второй матрицы  =  5.653090536812564e-10
относительная погрешность для первой матрицы =  0.0
относительная погрешность для второй матрицы =  5.056277889309467e-11


Вычисление обратной матрицы, проверка, нахождение числа обусловленности:

In [ ]:
def reverse(A):
    X = []
    A_1 = np.copy(A)
    x = np.zeros(A.shape[1])
    for i in range(A.shape[1]):
        x[i] = 1
        A = np.copy(A_1)
        X.append(solvehouse(A,x))
        x[i] = 0
    return X

X_1=np.transpose(reverse(A_1))
print(X_1@A_1)
X_2=np.transpose(reverse(A_2))
print(X_2@A_2)
print('Число обусловленности первой матрицы = ', np.linalg.norm(A_1)*np.linalg.norm(X_1))
print('Число обусловленности второй матрицы = ', np.linalg.norm(A_2)*np.linalg.norm(X_2))

[[ 1.00000000e+00 -3.23336039e-17  6.40073092e-19  1.22048429e-17]
 [ 1.10102434e-18  1.00000000e+00  6.65569211e-18 -7.62678253e-18]
 [-3.53684296e-18  8.52490620e-18  1.00000000e+00  1.51369306e-18]
 [-2.16266504e-17 -8.09968664e-19  8.57430559e-18  1.00000000e+00]]
[[ 9.99999997e-01  2.72837686e-09 -1.42513178e-08 -1.87661950e-08]
 [-3.20297155e-09  1.00000000e+00 -1.47378063e-08 -1.92941394e-08]
 [-1.90943142e-11  8.18037775e-12  1.00000000e+00 -3.80502818e-11]
 [-3.73903774e-11  8.28654692e-12  7.94958190e-12  1.00000000e+00]]
Число обусловленности первой матрицы =  5.665774461679196
Число обусловленности второй матрицы =  74587201.49508478


Сравнение с методом Гаусса:

In [ ]:
def gauss_method(matrix, vector):
    n = len(matrix)
    for i in range(n):
        max_el = abs(matrix[i][i])
        max_row = i
        for k in range(i + 1, n):
            if abs(matrix[k][i]) > max_el:
                max_el = abs(matrix[k][i])
                max_row = k

        matrix[i], matrix[max_row] = matrix[max_row], matrix[i]
        vector[i], vector[max_row] = vector[max_row], vector[i]

        for k in range(i + 1, n):
            c = -matrix[k][i] / matrix[i][i]
            for j in range(i, n):
                if i == j:
                    matrix[k][j] = 0
                else:
                    matrix[k][j] += c * matrix[i][j]
            vector[k] += c * vector[i]

    x = [0 for _ in range(n)]
    for i in range(n - 1, -1, -1):
        x[i] = vector[i] / matrix[i][i]
        for k in range(i - 1, -1, -1):
            vector[k] -= matrix[k][i] * x[i]
    return x
A_1 = np.array([[ 127.8000,8.0300,1.4000,-2.3600],
      [0.2700,136.4000,-0.1600,-4.5500],
     [-3.8400,5.3700,-111.0000,1.5600],
     [-6.5300,6.7200,2.8800,47.2000]])
b_1 = np.array([-1008.64, 516.62, 394.56, 353.68])
x_1 = np.array([-8,4,-3,6], dtype = 'float64')

A_2 = np.array([[3.8970,-3.8940,19.0620,27.2580],
     [29.1600,-29.1570,158.9520,198.7160],
      [0.9720,-0.9720,4.7610,6.8040],
      [2.9160,-2.9160,16.5900,19.6430]])
b_2 = np.array([-17.2050, -426.2380, -4.3470, -55.3360])
x_2 = np.array([13,14,-15,10], dtype = 'float64')

print(gauss_method(A_1,b_1))
print(gauss_method(A_2,b_2)) #не решает


[-8.0, 4.0, -3.0000000000000004, 6.0]
[nan, nan, inf, inf]


<ipython-input-10-7d9790486cc9>:25: RuntimeWarning: divide by zero encountered in scalar divide
  x[i] = vector[i] / matrix[i][i]
<ipython-input-10-7d9790486cc9>:27: RuntimeWarning: invalid value encountered in scalar subtract
  vector[k] -= matrix[k][i] * x[i]
